In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
import numpy as np
import itertools

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def load_test_data(loc, date, b1, b2):
    x = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{date}.npy")[:,::8,::8].flatten()[:,None]
    x = np.concatenate((x, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{date}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
    y = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{date}.npy")[:,::2,::2].flatten(),0,40)
    
    return x, y


def load_train_data(loc, dates, b1, b2):
    x = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{dates[0]}.npy")[:,::8,::8].flatten()[:,None]
    x = np.concatenate((x, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{dates[0]}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
    y = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{dates[0]}.npy")[:,::2,::2].flatten(),0,40)
    
    for d in dates[1:]:
        xi = np.load(f"/data/GPM_HIM8/exp_paper/X_B{b1}_{loc}_{d}.npy")[:,::8,::8].flatten()[:,None]
        xi = np.concatenate((xi, np.load(f"/data/GPM_HIM8/exp_paper/X_B{b2}_{loc}_{d}.npy")[:,::8,::8].flatten()[:,None]), axis=1)
        x = np.concatenate((x,xi), axis=0)
        yi = np.clip(np.load(f"/data/GPM_HIM8/exp_paper/Y_{loc}_{d}.npy")[:,::2,::2].flatten(),0,40)
        y = np.concatenate((y,yi), axis=0)
        
    return x, y

In [4]:
dates = ["201811","201812","201901","201902"]

print("model,mse,f1_01,f1_1")

for loc in ["SYD","NT","WA"]:
    for i, d in enumerate(dates):
        for bands in itertools.combinations([8,9,10,11,12,13,14,15,16], 2):
            x_train, y_train = load_train_data(loc, dates[:i]+dates[i+1:], bands[0], bands[1])
            lr = LinearRegression(n_jobs=8)
            lr.fit(x_train, y_train)
            
            x_test, y_test = load_test_data(loc, d, bands[0], bands[1])
            y_hat = lr.predict(x_test)
            
            print(f"{loc}_{d}_b{bands[0]}_b{bands[1]},{np.mean(np.square(y_hat-y_test))},{f1_score(y_hat>.2,y_test>.2)},{f1_score(y_hat>1.,y_test>1.)}")

model,mse,f1_01,f1_1
SYD_201811_b8_b9,0.6739554405212402,0.3183863038521442,0.22196870591691115
SYD_201811_b8_b10,0.6598267555236816,0.37139836971451035,0.32812235800476525
SYD_201811_b8_b11,0.6651725769042969,0.3373832470086302,0.29889376140154367
SYD_201811_b8_b12,0.7122485637664795,0.2000275094026425,0.07461158349285492
SYD_201811_b8_b13,0.6663333773612976,0.3354517608766296,0.2887833339993948
SYD_201811_b8_b14,0.6677514314651489,0.33896789653078085,0.27487941682936406
SYD_201811_b8_b15,0.6675967574119568,0.34959710359362184,0.2774606140290176
SYD_201811_b8_b16,0.6689785718917847,0.35085159955658307,0.3123190147262825
SYD_201811_b9_b10,0.6713860630989075,0.32813583493713866,0.3451087310280954
SYD_201811_b9_b11,0.6545069217681885,0.4097769530537968,0.19559563416495626
SYD_201811_b9_b12,0.6771712899208069,0.2922649959475744,0.3011301532574013
SYD_201811_b9_b13,0.655525803565979,0.40592032441444154,0.19693092737760687
SYD_201811_b9_b14,0.6571531295776367,0.40276501809973025,0.186150420

/home/pl5189/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


SYD_201812_b8_b12,1.2888973951339722,0.4101375721259747,0.0
SYD_201812_b8_b13,1.2680994272232056,0.4437489439315128,0.06821289485071398
SYD_201812_b8_b14,1.2705636024475098,0.4415158058129659,0.047666013527434034
SYD_201812_b8_b15,1.2702125310897827,0.44082108761003547,0.054044144926650634
SYD_201812_b8_b16,1.2636456489562988,0.4453576159150611,0.11872628196868493
SYD_201812_b9_b10,1.2449897527694702,0.44557673771746537,0.3412225740031439
SYD_201812_b9_b11,1.2674198150634766,0.4535509667145553,0.1057913725093983
SYD_201812_b9_b12,1.2684426307678223,0.3875476463957183,0.10987239758226996
SYD_201812_b9_b13,1.2681846618652344,0.4509052222767187,0.09864252244472946
SYD_201812_b9_b14,1.2696633338928223,0.44702619001055305,0.0888993074054342
SYD_201812_b9_b15,1.270636796951294,0.4452910164946056,0.08323348168435467
SYD_201812_b9_b16,1.2648375034332275,0.4501753786945507,0.12081367532726428
SYD_201812_b10_b11,1.2524943351745605,0.4473189985687483,0.26442536190711025
SYD_201812_b10_b12,1.25281

/home/pl5189/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


SYD_201812_b11_b12,1.2854208946228027,0.47331750933684535,0.0


...

/home/pl5189/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


WA_201902_b14_b15,0.04325979948043823,0.377610257108906,0.0


/home/pl5189/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


WA_201902_b14_b16,0.042614731937646866,0.27863033215411176,0.0
WA_201902_b15_b16,0.04265226051211357,0.2818884931663148,0.0


/home/pl5189/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
